In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
netflix_shows=pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
netflix_shows.head(10)

In [ ]:
netflix_shows.shape              # let's see how big is the dataset

Quite big dataset,good for having valuable insights and also for training models.

In [ ]:
# check null values of different columns
netflix_shows.isnull().sum()

In [ ]:
# Analyse counts of different ratings  
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
plt.figure(figsize=(14,8))
sns.countplot(x="rating",data=netflix_shows)

TV-MA(Matured Audiences),TV-14(for people above 14 years) are the most rated tv shows,which indicates there are a lot of matured contents,i.e majority of them are for people above 18 years of age.

In [ ]:
netflix_shows["date_added"]=netflix_shows["date_added"].fillna(method='ffill')
netflix_shows["date_added"].isnull().sum()     

Since very low number of values were missing,we just used ffill to fillup the missing values.

In [ ]:
year=netflix_shows["date_added"].to_list()  # convert to list for extracting the year added

In [ ]:
for i in range(len(year)):
    netflix_shows["year_added"].iloc[i]=year[i].split(',')[1]

In [ ]:
netflix_shows.head()

In [ ]:
import numpy as np
netflix_shows["year_added"]=netflix_shows["year_added"].astype(np.int)

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(14,8))
sns.countplot(x="type",hue="year_added",data=netflix_shows)

There is an overall balanced addition of tv-shows and movies,with 2019 the year being when most number of shows are added.

In [ ]:
import matplotlib.pyplot as plt
values=dict(netflix_shows["type"].value_counts())
labels=values.keys()
sizes=values.values()
fig1, ax1 = plt.subplots()
ax1.pie(sizes,labels=labels, shadow=True,autopct='%1.1f%%',startangle=90)
plt.show()

In [ ]:
# Let's check some oldest movies and TV shows on netflix
netflix_shows[["title","release_year"]].groupby(["release_year"]).min().head(10)


In [ ]:
# Let's focus on the distribution of duration of movies
def plot_movie_duration():
    netflix_duration=netflix_shows["duration"].copy()
    index=[]
    for i in range(len(netflix_shows["duration"])):
        if netflix_shows["type"].iloc[i]=='TV Show':
            index.append(i)
        
        
    netflix_duration=netflix_duration.drop(index)
    for i in range(len(netflix_duration)):
        x=netflix_duration.iloc[i].split(' ')[0]
        netflix_duration.iloc[i]=x
        
    plt.figure(figsize=(12,8))
    sns.distplot(netflix_duration)

plot_movie_duration()

In [ ]:
def plot_series_duration():
    netflix_tv_show_duration=netflix_shows["duration"].copy()
    index=[]
    for i in range(len(netflix_shows["duration"])):
        if netflix_shows["type"].iloc[i]=='Movie':
            index.append(i)
        
        
    netflix_tv_show_duration=netflix_tv_show_duration.drop(index)
    for i in range(len(netflix_tv_show_duration)):
        x=netflix_tv_show_duration.iloc[i].split(' ')[0]
        netflix_tv_show_duration.iloc[i]=x
        
    plt.figure(figsize=(12,8))
    sns.countplot(netflix_tv_show_duration)

plot_series_duration()

I don't know why the plot showing(10,15) before 2.      :<

However,there are many TV-shows with 1 season.

In [ ]:
# Now see the top categories of shows
from collections import Counter 


Genres=netflix_shows["listed_in"].copy()
categories=[]
for i in range(len(Genres)):
    elements=Genres.iloc[i].split(",")
    for j in range(len(elements)):
        categories.append(elements[j])
count_categories=Counter(categories)
#print(count_categories)

count_categories=count_categories.most_common()   # for sorting values
keys=[count_categories[i][0] for i in range(len(count_categories))]
values=[count_categories[i][1] for i in range(len(count_categories))]


top_10_keys=keys[0:10]
top_10_values=values[0:10]
plt.figure(figsize=(21,10))
sns.barplot(top_10_keys,top_10_values)
plt.title("Top 10 movie-series categories")

Lets do some exploration on distribution of netflix contents across different countries.

In [ ]:
country=netflix_shows["country"].copy()
country=country.dropna(axis=0,how='all')
country_list=[]
for i in range(len(country)):
    elements=country.iloc[i].replace(" ","").split(",")
    for j in range(len(elements)):
        country_list.append(elements[j])
country_dict=Counter(country_list)
#print(country_dict)
country_dict=country_dict.most_common()

keys=[country_dict[i][0] for i in range(len(country_dict))]
values=[country_dict[i][1] for i in range(len(country_dict))]

top_20_countries=keys[:20]
top_20_values=values[:20]

plt.style.use('Solarize_Light2')
plt.figure(figsize=(14,10))
sns.barplot(top_20_values,top_20_countries)

USA tops the chart with most content,followed by India and UK.Is this because Netflix is an American Multinational Company with majority of their users based on USA?....Probably,Yes.

Lets do something on top actors in the top 5 countries with most content.

In [ ]:
actors_country=netflix_shows[['cast','country']]
actors_country=actors_country.dropna(axis=0,how='any')

top_actors_USA=[]
top_actors_India=[]
top_actors_UK=[]
top_actors_Canada=[]
top_actors_France=[]

for i in range(actors_country.shape[0]):
    country = list(actors_country['country'].iloc[i].replace(" ", "").split(","))
    actors= list(actors_country['cast'].iloc[i].split(","))
    for j in range(len(actors)):
        for k in range(len(country)):
            actors[j]=actors[j].strip()
            if country[k]=='UnitedStates':
                top_actors_USA.append(actors[j])
            if country[k]=='India':
                top_actors_India.append(actors[j])
            if country[k]=='UnitedKingdom':
                top_actors_UK.append(actors[j])
            if country[k]=='Canada':
                top_actors_Canada.append(actors[j])
            if country[k]=='France':
                top_actors_France.append(actors[j])



In [ ]:
def plot_top_actors(top_actors):
    top_actors=Counter(top_actors)
    top_actors=top_actors.most_common()
    
    keys=[top_actors[i][0] for i in range(len(top_actors))]
    values=[top_actors[i][1] for i in range(len(top_actors))]

    top_20_keys=keys[:20]
    top_20_values=values[:20]
    
    plt.style.use('classic')
    plt.figure(figsize=(10,6))
    sns.barplot(top_20_values,top_20_keys)
    

plot_top_actors(top_actors_India)
plt.title("Top Indian actors")

#plot_top_actors(top_actors_USA)

In [ ]:
plot_top_actors(top_actors_USA)
plt.title("Top American actors")

In [ ]:
plot_top_actors(top_actors_UK)
plt.title("Top English actors")

In [ ]:
plot_top_actors(top_actors_Canada)
plt.title("Top Canadian actors")

In [ ]:
plot_top_actors(top_actors_France)
plt.title("Top French actors")

So,that's all for the actors.That was one hell of an analysis.Let us move on to the directors.

In [ ]:
directors_country=netflix_shows[['director','country']]
directors_country=directors_country.dropna(axis=0,how='any')

top_director_USA=[]
top_director_INDIA=[]
top_director_UK=[]
top_director_CANADA=[]
top_director_FRANCE=[]

for i in range(directors_country.shape[0]):
    country = list(directors_country['country'].iloc[i].replace(" ", "").split(","))
    directors= list(directors_country['director'].iloc[i].split(","))
    for j in range(len(directors)):
        for k in range(len(country)):
            directors[j]=directors[j].strip()
            if country[k]=='UnitedStates':
                top_director_USA.append(directors[j])
            if country[k]=='India':
                top_director_INDIA.append(directors[j])
            if country[k]=='UnitedKingdom':
                top_director_UK.append(directors[j])
            if country[k]=='Canada':
                top_director_CANADA.append(directors[j])
            if country[k]=='France':
                top_director_FRANCE.append(directors[j])


In [ ]:
def plot_top_directors(top_directors):
    top_directors=Counter(top_directors)
    top_directors=top_directors.most_common()
    
    keys=[top_directors[i][0] for i in range(len(top_directors))]
    values=[top_directors[i][1] for i in range(len(top_directors))]

    top_20_keys=keys[:20]
    top_20_values=values[:20]
    
    plt.style.use('classic')
    plt.figure(figsize=(10,6))
    sns.barplot(top_20_values,top_20_keys)
    

plot_top_actors(top_director_INDIA)
plt.title("Top Indian directors")


In [ ]:
plot_top_actors(top_director_USA)
plt.title("Top American directors")

In [ ]:
plot_top_actors(top_director_UK)
plt.title("Top English directors")

In [ ]:
plot_top_actors(top_director_CANADA)
plt.title("Top Canadian directors")

In [ ]:
plot_top_actors(top_director_FRANCE)
plt.title("Top French directors")

That's all.As a beginner in EDA,i tried to do my best.If you like it,upvote this.Also let me know how I can improve the analysis and visualization better.